In [1]:
#| include: false

import duckdb
import pandas as pd
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});

,Count
0,87


# Фільтрація даних

Фільтрація даних дозволяє включати в результуючий запит не всі рядки вихідної таблиці, а лише ті, що відповідають заздалегідь заданій умові.

## WHERE

В SQL для фільтрації даних використовується оператор `WHERE`. Після оператора `WHERE` вказується логічний вираз, результат якого визначає, чи буде рядок включений до результуючої таблиці. Якщо умова виявляється істинною (`TRUE`), то рядок включається до результату, якщо хибним (`FALSE`) — рядок виключається.

Таким чином, кожен рядок у таблиці проходить перевірку на відповідність певній умові, і в результаті цих перевірок формується таблиця, над якою проводяться операції, зазначені в блоці `SELECT`.

Оператор `WHERE` та логічний вираз вказуються після блоку `FROM`:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 >= 0
```

Наприклад, у результаті зазначеного вище запиту у вибірці попадуть лише записи з невід'ємними значеннями в колонці `column_2`.

У свою чергу оператори `ORDER BY` і `LIMIT` записуються вже після оператора `WHERE`. Якщо додати їх у приклад вище, запит виглядатиме так:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 >= 0
ORDER BY column_1
LIMIT 100
```

У результаті ми спочатку відфільтруємо необхідні нам рядки, потім виберемо стовпчики, виділені в `SELECT`, а потім сортуємо результуючу таблицю, обмеживши число виведених записів.

Таким чином порядок запису відомих нам на поточний момент ключових слів виглядає так:

1. `SELECT`
2. `FROM`
3. `WHERE`
4. `ORDER BY`
5. `LIMIT`

Знову звернемо увагу, що порядок виконання відрізняється від того, в якій послідовності вони вказуються в запиті:

1. Спочатку виконується оператор `FROM` – відбувається вибір потрібної таблиці.
2. Далі `WHERE` - відфільтровуються рядки, що відповідають умові.
3. Потім `SELECT` - відбираються зазначені стовпці та застосовуються функції.
4. Потім `ORDER BY` - проводиться сортування результуючої таблиці.
5. І наприкінці `LIMIT` — обмежується кількість записів, що виводяться.

Іншими словами, в результаті виконання запиту спочатку відбувається підготовка таблиці до роботи, а потім над нею виконуються різні операції.

::: {.callout-note}
Докладніше про оператор `WHERE` можна за [посиланням](https://duckdb.org/docs/sql/query_syntax/where).
:::

::: {#exr-sql-filter-01}
Напишіть SQL-запит до таблиці `products` та виведіть всю інформацію про товари, ціна яких не перевищує 100 одиниць. Результат відсортуйте за зростанням id товару.

Поля у результуючій таблиці: `product_id`, `name`, `price`

**Рішення:**
:::

In [2]:
%%sql
SELECT 
    product_id,
    name,
    price
FROM   products
WHERE  price <= 100
ORDER BY product_id
LIMIT  10;

,product_id,name,price
0,2,green tea bags,50.0
1,3,still water,80.0
2,4,lollipops,46.0
3,5,coffee 3 in 1,15.0
4,6,crackers,25.0
5,8,drying,30.0
6,9,black leaf tea,84.0
7,10,seeds,12.0
8,14,mayonnaise,60.0
9,18,ketchup,58.0


Фільтрувати дані в таблицях можна не тільки по полях з числовими значеннями, але і по полях зі значеннями, представленими у вигляді тексту:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 = 'text'
```

У прикладі вище в результуючу таблицю потраплять лише рядки, значення в яких повністю збігаються із зазначеним у `WHERE` рядком `'text'`.

При порівнянні рядків також допускається використовувати нерівності:

```{.sql}
SELECT column_1, column_2
FROM table
WHERE column_2 > 'text'
```

Втім, така операція використовується рідше, оскільки не цілком очевидно, що означає «один рядок більше за інший».

Насправді порядок сортування даних рядкового типу зазвичай визначається заздалегідь встановленими правилами сортування, у яких значення мають довжина рядка в символах, порядок букв відповідно до алфавіту, наявність особливих символів, регістр тощо. Ми не будемо докладно зупинятись на цій темі — за бажання про це можна додатково прочитати у документації.

::: {.callout-note}
Докладніше про правила сортування даних рядкового типу можна за [посиланням](https://duckdb.org/docs/sql/expressions/collations).
:::

::: {#exr-sql-filter-02}
Виберіть користувачів жіночої статі з таблиці `users`. Виведіть лише id цих користувачів. Результат відсортуйте за зростанням id.

Додайте в запит оператор `LIMIT` і виведіть лише `10` перших ID з відсортованого списку.

Поле у результуючій таблиці: `user_id`

**Рішення:**

:::

In [3]:
%%sql
SELECT 
    user_id
FROM   
    users
WHERE  
    sex = 'female'
ORDER BY 
    user_id 
LIMIT 10

,user_id
0,7
1,131
2,202
3,246
4,346
5,407
6,474
7,517
8,591
9,686
